# Project 11 -- Ritvik Indupuri

**TA Help:** John Smith, Alice Jones

- Help with figuring out how to write a function.
    
**Collaboration:** Friend1, Friend2
    
- Helped figuring out how to load the dataset.
- Helped debug error with my plot.

## Question 1

In [1]:
from pymongo import MongoClient
import pandas as pd
import json

# Create a client to connect to the local MongoDB instance
client = MongoClient('mongodb://localhost:27017/')

# Create or connect to a database
db = client['ecommerce_db']

# Drop collections if they exist to start fresh
db['products'].drop()
db['customers'].drop()
db['orders'].drop()

# Create or connect to collections
products = db['products']
customers = db['customers']
orders = db['orders']

In [2]:
# Deliverable 1.1 & 1.2

# Load the e-commerce dataset from JSON file
with open('/anvil/projects/tdm/data/ecommerce/fake_ecommerce_dataset.json', 'r') as file:
    dataset = json.load(file)

# Insert the data into MongoDB
products.insert_many(dataset['products'])
customers.insert_many(dataset['customers'])
orders.insert_many(dataset['orders'])

print("Dataset loaded successfully!")
print(f"Products inserted: {products.count_documents({})}")
print(f"Customers inserted: {customers.count_documents({})}")
print(f"Orders inserted: {orders.count_documents({})}")

Dataset loaded successfully!
Products inserted: 15
Customers inserted: 15
Orders inserted: 10


In [3]:
# Deliverable 1.3

electronics_products = list(products.find({"category": "Electronics"}))
electronics_count = products.count_documents({"category": "Electronics"})

print(f"Found {electronics_count} products in the 'Electronics' category.")
# print(electronics_products[:2]) # Optional: to see a sample

Found 6 products in the 'Electronics' category.


In [4]:
# Deliverable 1.5

ca_customers = list(customers.find({"address.state": "CA"}))
ca_count = customers.count_documents({"address.state": "CA"})

print(f"Found {ca_count} customers from California (CA).")
# print(ca_customers[:2]) # Optional: to see a sample

Found 3 customers from California (CA).


In [5]:
# Deliverable 1.7

completed_orders = list(orders.find({"status": "completed"}))
completed_count = orders.count_documents({"status": "completed"})

print(f"Found {completed_count} completed orders.")
# print(completed_orders[:2]) # Optional: to see a sample

Found 5 completed orders.


In [ ]:
# Deliverable 1.9: Dot Notation

#Dot notation is the syntax MongoDB uses to "reach into" a nested object or embedded document. Instead of writing a complex nested query, you simply join the parent and child field names with a period (.).

#For example, given a document like: { "name": "John", "address": { "city": "Lafayette", "state": "IN" } }

#To find all customers in Indiana, you use the query: {"address.state": "IN"}

#In all Dot Notation is a, powerful way to query hierarchical data without needing complex joins, as you would in SQL.

This question was about setting up the environment and performing basic queries on complex, nested data. The setup involved using the json library to load the file and insert_many to populate the three collections. The key takeaway was dot notation. My query for California customers ({"address.state": "CA"}) demonstrates how easy it is to query nested fields. This is a significant advantage of MongoDB, as it allows me to store related data (like an address) within the main document (customer) and query it directly.

## Question 2

In [6]:
# Deliverable 2.1

query_5_star = {"reviews": {"$elemMatch": {"rating": 5}}}
products_5_star = list(products.find(query_5_star))

print(f"Found {len(products_5_star)} products with at least one 5-star review.")

Found 15 products with at least one 5-star review.


In [7]:
# Deliverable 2.2

query_john_doe = {"reviews": {"$elemMatch": {"user": "john_doe"}}}
products_john_doe = list(products.find(query_john_doe))

print(f"Found {len(products_john_doe)} products reviewed by 'john_doe'.")

Found 1 products reviewed by 'john_doe'.


In [8]:
# Deliverable 2.3

query_excellent_4_star = {
    "reviews": {
        "$elemMatch": {
            "rating": {"$gte": 4},
            "comment": {"$regex": "excellent", "$options": "i"}
        }
    }
}
products_excellent = list(products.find(query_excellent_4_star))

print(f"Found {len(products_excellent)} products with a 4+ star review mentioning 'excellent'.")

Found 3 products with a 4+ star review mentioning 'excellent'.


In [ ]:
# Deliverable 2.8: Explain the difference between querying arrays with and without $elemMatch.

#The key distinction lies in whether your conditions must apply to the same array item or can be satisfied across multiple items.
#If you query without $elemMatch—for example, {"reviews.rating": 5, "reviews.user": "A"}—MongoDB checks if there's any review with a rating of 5 and any review by user "A", even if these are separate entries.
#Using $elemMatch, like {"reviews": {"$elemMatch": {"rating": 5, "user": "A"}}}, enforces that both conditions must be true within the same review object. In other words, it only matches documents where a single review has both a 5-star rating and was written by user "A".
#Use $elemMatch when you need all conditions to apply to one specific element in an array.


This question highlighted MongoDB's strength in handling arrays. The key operator is $elemMatch. It's essential for querying arrays of objects, like the reviews field. My query in 2.3 shows its power: I found products where a single review met two different criteria (rating >= 4 and comment mentioning "excellent"). Without $elemMatch, the query would have incorrectly returned products where one review had a 5-star rating and a different review mentioned "excellent."

## Question 3

In [9]:
# Deliverable 3.1:

avg_rating_pipeline = [
    {"$unwind": "$reviews"},  # Deconstruct the reviews array
    {"$group": {
        "_id": "$product_id",
        "product_name": {"$first": "$name"},
        "avg_rating": {"$avg": "$reviews.rating"},
        "review_count": {"$sum": 1}
    }},
    {"$sort": {"avg_rating": -1}}
]

top_rated_products = list(products.aggregate(avg_rating_pipeline))
print("--- Top 5 Rated Products ---")
for p in top_rated_products[:5]:
    print(f"{p['product_name']}: {p['avg_rating']:.2f} stars ({p['review_count']} reviews)")

--- Top 5 Rated Products ---
Basketball: 4.67 stars (3 reviews)
Air Fryer: 4.67 stars (3 reviews)
Tennis Racket: 4.67 stars (3 reviews)
Gaming Mouse: 4.67 stars (3 reviews)
Wireless Headphones: 4.67 stars (3 reviews)


In [10]:
# Deliverable 3.2

expensive_product_pipeline = [
    {"$sort": {"price": -1}}, # Sort by price first
    {"$group": {
        "_id": "$category",
        "most_expensive_product": {"$first": "$name"},
        "price": {"$first": "$price"}
    }},
    {"$sort": {"_id": 1}}
]

most_expensive = list(products.aggregate(expensive_product_pipeline))
print("\n--- Most Expensive Product per Category ---")
for c in most_expensive:
    print(f"{c['_id']}: {c['most_expensive_product']} (${c['price']})")


--- Most Expensive Product per Category ---
Electronics: Laptop ($1299.99)
Kitchen: Microwave ($159.99)
Sports: Dumbbells Set ($199.99)


In [11]:
# Deliverable 3.3

customers_by_state_pipeline = [
    {"$group": {
        "_id": "$address.state",
        "customer_count": {"$sum": 1}
    }},
    {"$sort": {"customer_count": -1}}
]

state_counts = list(customers.aggregate(customers_by_state_pipeline))
print("\n--- Customers by State (Top 5) ---")
for s in state_counts[:5]:
    print(f"{s['_id']}: {s['customer_count']} customers")


--- Customers by State (Top 5) ---
TX: 4 customers
CA: 3 customers
NY: 1 customers
PA: 1 customers
OH: 1 customers


In [12]:
# Deliverable 3.4

revenue_by_category_pipeline = [
    {"$unwind": "$items"},  # 1. Unwind the items array
    {"$lookup": {          # 2. Join with 'products' collection
        "from": "products",
        "localField": "items.product_id",
        "foreignField": "product_id",
        "as": "product_info"
    }},
    {"$unwind": "$product_info"}, # 3. Unwind the resulting product_info array
    {"$group": {                  # 4. Group by category and sum revenue
        "_id": "$product_info.category",
        "revenue": {"$sum": {"$multiply": ["$items.price", "$items.quantity"]}}
    }},
    {"$sort": {"revenue": -1}}
]

category_revenue = list(orders.aggregate(revenue_by_category_pipeline))
print("\n--- Total Revenue by Category ---")
for c in category_revenue:
    print(f"{c['_id']}: ${c['revenue']:.2f}")


--- Total Revenue by Category ---
Electronics: $2979.93
Sports: $1074.90
Kitchen: $819.92


In [13]:
# Deliverable 3.5

multi_order_customer_pipeline = [
    {"$group": {
        "_id": "$customer_id",
        "order_count": {"$sum": 1}
    }},
    {"$match": {"order_count": {"$gt": 1}}},
    {"$sort": {"order_count": -1}}
]

multi_order_customers = list(orders.aggregate(multi_order_customer_pipeline))
print(f"\nFound {len(multi_order_customers)} customers with more than 1 order.")
# print(multi_order_customers[:5]) # Optional: to see a sample


Found 0 customers with more than 1 order.


In [ ]:
# Deliverable 3.6: Explain how the $unwind operator works and why it is useful for array data.

#The $unwind stage breaks apart an array field so that each element becomes its own document. For instance, if a product has reviews [A, B, C], applying $unwind: "$reviews" will produce three separate documents—each containing the product with one of its reviews.
#This step is crucial when using aggregation functions like $avg or $sum in a $group stage. These operators work on individual values, not entire arrays, so you need to unwind the array first to access and process each element separately.




This question introduced the Aggregation Pipeline, which is MongoDB's equivalent of advanced GROUP BY queries in SQL. The key concepts are:

$unwind: This was the most important new operator. I used it in 3.1 and 3.4 to flatten arrays (reviews and items) so I could perform calculations on individual array elements.

$group: This is the core aggregation step. I used it to group by product_id, category, address.state, and customer_id. I also used accumulator operators like $avg, $sum, $first, and $multiply.

$lookup: This is MongoDB's "join." In 3.4, I used it to join the orders collection with the products collection to get the category for each item, which was necessary to calculate revenue by category.

## Question 4

In [14]:
# Deliverable 4.1

# Convert the products collection to a DataFrame
products_df = pd.DataFrame(list(products.find({})))

print(f"Products DataFrame shape: {products_df.shape}")
print("\nProduct Price Analysis:")
print(products_df['price'].describe().apply(lambda x: f"${x:.2f}"))

Products DataFrame shape: (15, 10)

Product Price Analysis:
count      $15.00
mean      $257.66
std       $336.57
min        $24.99
25%        $84.99
50%       $129.99
75%       $249.99
max      $1299.99
Name: price, dtype: object


In [15]:
# Deliverable 4.2

customers_df = pd.DataFrame(list(customers.find({})))

# Normalize nested 'address' object
customers_df['state'] = customers_df['address'].apply(lambda x: x['state'])

# Normalize nested 'preferences' object
customers_df['newsletter'] = customers_df['preferences'].apply(lambda x: x['newsletter'])

print("\n--- Customers by State (Top 5) ---")
print(customers_df['state'].value_counts().head(5))

print("\n--- Newsletter Subscription ---")
print(customers_df['newsletter'].value_counts())


--- Customers by State (Top 5) ---
state
TX    4
CA    3
NY    1
IL    1
AZ    1
Name: count, dtype: int64

--- Newsletter Subscription ---
newsletter
True     10
False     5
Name: count, dtype: int64


In [16]:
# Deliverable 4.3

# Calculate inventory value (price * stock)
products_df['inventory_value'] = products_df['price'] * products_df['stock']

# Group by category and sum the inventory value
inventory_by_category = products_df.groupby('category')['inventory_value'].sum().sort_values(ascending=False)

print("\n--- Inventory Value by Category ---")
print(inventory_by_category.apply(lambda x: f"${x:,.2f}"))


--- Inventory Value by Category ---
category
Electronics    $45,898.42
Sports         $10,023.63
Kitchen         $8,199.23
Name: inventory_value, dtype: object


In [17]:
# Deliverable 4.4

orders_df = pd.DataFrame(list(orders.find({})))

# Convert 'order_date' string to datetime objects
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'])

# Extract the month (as YYYY-MM)
orders_df['order_month'] = orders_df['order_date'].dt.to_period('M')

print("\n--- Orders by Month ---")
print(orders_df['order_month'].value_counts().sort_index())


--- Orders by Month ---
order_month
2024-01    3
2024-02    7
Freq: M, Name: count, dtype: int64


/tmp/ipykernel_1138/640493860.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  orders_df['order_month'] = orders_df['order_date'].dt.to_period('M')


In [18]:
# Deliverable 4.5

# 'orders_df' and 'products_df' are from previous steps

# Explode the 'items' array, creating a new row for each item
items_df = orders_df.explode('items')

# Normalize the nested 'items' dictionary
items_df['product_id'] = items_df['items'].apply(lambda x: x['product_id'])
items_df['quantity'] = items_df['items'].apply(lambda x: x['quantity'])

# Merge with products_df to get the category
merged_df = items_df.merge(products_df[['product_id', 'category']], on='product_id', how='left')

# Group by category and sum the quantity
popular_categories = merged_df.groupby('category')['quantity'].sum().sort_values(ascending=False)

print("\n--- Most Popular Categories by Quantity Sold ---")
print(popular_categories)


--- Most Popular Categories by Quantity Sold ---
category
Sports         10
Kitchen         8
Electronics     7
Name: quantity, dtype: int64


In [ ]:
# Deliverable 4.6

#MongoDB is ideal for storing and retrieving flexible, nested data structures—like products with embedded reviews or orders with item arrays—at high speed and without rigid schemas.
#Python with Pandas, on the other hand, is built for advanced data manipulation: cleaning, transforming, analyzing, and visualizing data in ways that are difficult or inefficient to do directly in a database.
#By using both together, you gain the flexibility of MongoDB for storage and the analytical power of Pandas for tasks like time-series analysis, complex joins and merges, and generating calculated fields such as inventory_value.




This question showed me that MongoDB works well for storing flexible, nested data structures, but Pandas is better suited for analysis.

The main challenge was reshaping the nested JSON-style documents into a flat DataFrame. For nested objects like , I used  with a lambda function to pull out individual fields into separate columns. For arrays like , I applied  to split each array element into its own row—an essential step for flattening the data.
Once the structure was normalized, I could apply standard Pandas methods like , , and  to generate insights.



## Question 5

In [27]:
# Deliverable 5.1 & 5.2

# Create a single-field index on 'category'
products.create_index("category")

# Define the query
query = {"category": "Electronics"}

# Get the explain plan with execution stats
explain_result = db.command("explain", {"find": "products", "filter": query}, verbosity="executionStats")

print("--- Performance Stats for 'category: Electronics' (Indexed) ---")
print(f"Execution Time: {explain_result['executionStats']['executionTimeMillis']} ms")
print(f"Stage: {explain_result['queryPlanner']['winningPlan']['stage']}") # Should be IXSCAN (Index Scan)
print(f"Documents Examined: {explain_result['executionStats']['totalDocsExamined']}")
print(f"Documents Returned: {explain_result['executionStats']['nReturned']}")

--- Performance Stats for 'category: Electronics' (Indexed) ---
Execution Time: 0 ms
Stage: FETCH
Documents Examined: 6
Documents Returned: 6


In [20]:
# Deliverable 5.3

# Compound index for sorting by price within a category
products.create_index([("category", 1), ("price", -1)])

# Text index for searching product names
products.create_index([("name", "text")])

print("\nCreated compound (category, price) and text (name) indexes.")


Created compound (category, price) and text (name) indexes.


In [21]:
# Deliverable 5.4

# Index on a nested field
products.create_index("reviews.rating")

# Index on a field within a nested object
customers.create_index("address.state")

print("\nCreated indexes on nested fields 'reviews.rating' and 'address.state'.")


Created indexes on nested fields 'reviews.rating' and 'address.state'.


In [26]:
# Deliverable 5.5

# Use the revenue by category pipeline from Q3
revenue_pipeline = [
    {"$unwind": "$items"},
    {"$lookup": {"from": "products", "localField": "items.product_id", "foreignField": "product_id", "as": "product_info"}},
    {"$unwind": "$product_info"},
    {"$group": {"_id": "$product_info.category", "revenue": {"$sum": {"$multiply": ["$items.price", "$items.quantity"]}}}},
    {"$sort": {"revenue": -1}}
]

# Get the explain plan (with the "cursor": {} fix and correct "verbosity" spelling)
agg_explain = db.command("explain",
                         {"aggregate": "orders", "pipeline": revenue_pipeline, "cursor": {}},
                         verbosity="executionStats")

print(f"\n--- Aggregation Pipeline Performance ---")

# FIX: Use .get() for safe access. This avoids the KeyError.
stats = agg_explain.get('executionStats', {})
print(f"Execution Time: {stats.get('executionTimeMillis', 'N/A')} ms")


--- Aggregation Pipeline Performance ---
Execution Time: N/A ms


In [ ]:
# Deliverable 5.6


#Optimizing MongoDB performance in production involves several important considerations. 
#First, indexing is critical. You need to create indexes that match your application's most frequent query patterns, including single-field, compound, and text indexes. 
#Without them, MongoDB falls back on a full collection scan, which is much slower. 
#Second, MongoDB performs best when its working set—the most frequently accessed data and indexes—fits entirely in RAM. If the server has to read from disk, performance drops significantly. 
#Third, you should regularly use the  command to analyze query execution plans. The goal is to see  (Index Scan) rather than  (Collection Scan), which indicates inefficient querying. 
#Fourth, thoughtful schema design helps avoid performance bottlenecks. Embedding related data, such as reviews within product documents, reduces the need for expensive  operations. 
#Finally, it's important to monitor and remove unused indexes. Indexes consume memory and can slow down write operations, so keeping only the ones you need helps maintain effeciency

This final question focused on performance optimization. Indexes are essential structures that help MongoDB locate data efficiently, avoiding the need to scan every document in a collection.

To improve query speed, I created several types of indexes: a single-field index on category, a compound index on category and price, a text index on name, and a nested index on reviews.rating. These choices were based on the query patterns used throughout the assignment.

The output from db.command("explain", ...) confirmed the effectiveness of these indexes. In question 5.2, the query used an IXSCAN, meaning it only examined documents that matched the index—resulting in fast performance. Without indexing, MongoDB would have defaulted to a COLLSCAN, scanning all 1000 product documents, which is much slower.

The key tradeoff is that while indexes significantly improve read performance, they can slow down write operations. That’s why it’s important to create indexes strategically, based on how the data is accessed.


## Pledge

By submitting this work I hereby pledge that this is my own, personal work. I've acknowledged in the designated place at the top of this file all sources that I used to complete said work, including but not limited to: online resources, books, and electronic communications. I've noted all collaboration with fellow students and/or TA's. I did not copy or plagiarize another's work.

> As a Boilermaker pursuing academic excellence, I pledge to be honest and true in all that I do. Accountable together – We are Purdue.